<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/NAT_AOCC_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --upgrade -q
import colab_env

In [ ]:
!pip install uv -q

In [ ]:
!uv pip install "nvidia-nat[all]"

In [9]:
import asyncio
import random
import os
import getpass
import time
from typing import Dict, Any, AsyncGenerator

import nest_asyncio
nest_asyncio.apply()

from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig
from nat.cli.type_registry import GlobalTypeRegistry
from pydantic import Field

# 1. Setup Environment
try:
    GlobalTypeRegistry.get()._registered_functions.clear()
    print("Function registry cleared")
except Exception:
    pass

if "NVIDIA_API_KEY" not in os.environ:
    os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter your NVIDIA API key: ")

# 2. Input Models
class WeatherInput(FunctionBaseConfig, name="weather_input"):
    flight_id: str = Field(..., description="Flight number")

class CrewInput(FunctionBaseConfig, name="crew_input"):
    flight_id: str = Field(..., description="Flight number")

class MaintenanceInput(FunctionBaseConfig, name="maintenance_input"):
    flight_id: str = Field(..., description="Flight number")

class PaxInput(FunctionBaseConfig, name="pax_input"):
    flight_id: str = Field(..., description="Flight number")

class FleetInput(FunctionBaseConfig, name="fleet_input"):
    flight_id: str = Field(..., description="Flight number")

class ATCInput(FunctionBaseConfig, name="atc_input"):
    flight_id: str = Field(..., description="Flight number")

class RecoveryInput(FunctionBaseConfig, name="recovery_input"):
    flight_id: str = Field(..., description="Flight number")
    requirement: str = Field(..., description="Resource to recover")

class CostInput(FunctionBaseConfig, name="cost_input"):
    flight_id: str = Field(..., description="Flight number")
    decision: str = Field(..., description="Final decision")
    delay_minutes: int = Field(default=0)
    pax_count: int = Field(default=0)

class ObservabilityInput(FunctionBaseConfig, name="observability_input"):
    flight_id: str = Field(..., description="Flight number")
    trace: Dict[str, Any] = Field(..., description="Execution data")
    latency_ms: float = Field(..., description="Total processing time")

class RefinerInput(FunctionBaseConfig, name="refiner_input"):
    decision: str = Field(..., description="Final decision")
    reason: str = Field(..., description="Reasoning")
    flight_id: str = Field(..., description="Flight number")
    cost: int = Field(default=0)

# 3. Agent Registrations
@register_function(config_type=WeatherInput)
async def register_weather(config, builder):
    async def logic(input_data):
        status = random.choice(["CLEAR", "THUNDERSTORMS", "FOG", "STRONG_WINDS"])
        return {"status": status, "severity": "HIGH" if status != "CLEAR" else "LOW"}
    yield logic

@register_function(config_type=CrewInput)
async def register_crew(config, builder):
    async def logic(input_data):
        hours = random.randint(1, 14)
        return {"ready": hours >= 5, "hours_left": hours}
    yield logic

@register_function(config_type=MaintenanceInput)
async def register_maintenance(config, builder):
    async def logic(input_data):
        status = random.choice(["GO", "AOG", "DEFERRED", "MEL"])
        return {"status": status}
    yield logic

@register_function(config_type=PaxInput)
async def register_pax(config, builder):
    async def logic(input_data):
        count = random.randint(0, 50)
        return {"count": count, "impact": "HIGH" if count > 20 else "LOW"}
    yield logic

@register_function(config_type=FleetInput)
async def register_fleet(config, builder):
    async def logic(input_data):
        return {"spare_ready": random.choice([True, False]), "sub_tail": f"N{random.randint(100, 999)}NV"}
    yield logic

@register_function(config_type=ATCInput)
async def register_atc(config, builder):
    async def logic(input_data):
        return {"delay": random.choice([0, 15, 45, 90])}
    yield logic

@register_function(config_type=RecoveryInput)
async def register_recovery(config, builder):
    async def logic(input_data):
        return {"success": random.random() < 0.4}
    yield logic

@register_function(config_type=CostInput)
async def register_cost(config, builder):
    async def logic(input_data):
        costs = {"PROCEED": 0, "DELAY": input_data.delay_minutes * 101, "SWAP AIRCRAFT": 5000, "CANCEL": 25000 + (input_data.pax_count * 350)}
        return {"total_cost": costs.get(input_data.decision, 0)}
    yield logic

@register_function(config_type=ObservabilityInput)
async def register_observability(config, builder):
    async def logic(input_data):
        print(f"[OBSERVABILITY] {input_data.flight_id} → Latency: {input_data.latency_ms:.2f}ms")
        return {"status": "Logged"}
    yield logic

@register_function(config_type=RefinerInput)
async def register_refiner(config, builder):
    async def logic(input_data):
        templates = {
            "CANCEL": f"CRITICAL: Flight CANCELLED. {input_data.reason}. Cost: ${input_data.cost:,}",
            "SWAP AIRCRAFT": f"OPS ALERT: Aircraft SWAP. {input_data.reason}. Cost: ${input_data.cost:,}",
            "DELAY": f"SCHEDULE UPDATE: Flight DELAYED. {input_data.reason}. Cost: ${input_data.cost:,}",
            "PROCEED": f"STATUS: On schedule. Cost: ${input_data.cost:,}"
        }
        return {"notification": templates.get(input_data.decision, input_data.reason)}
    yield logic

# 4. Orchestrator with Observability
async def run_full_aocc(flight_id: str):
    start_time = time.perf_counter()
    async with register_weather(None, None) as weather_logic, \
               register_crew(None, None) as crew_logic, \
               register_maintenance(None, None) as mx_logic, \
               register_pax(None, None) as pax_logic, \
               register_fleet(None, None) as fleet_logic, \
               register_atc(None, None) as atc_logic, \
               register_recovery(None, None) as recovery_logic, \
               register_cost(None, None) as cost_logic, \
               register_observability(None, None) as obs_logic, \
               register_refiner(None, None) as refiner_logic:

        # Step 1: Parallel Data Gathering
        w, p, a, c, m, f = await asyncio.gather(
            weather_logic(WeatherInput(flight_id=flight_id)),
            pax_logic(PaxInput(flight_id=flight_id)),
            atc_logic(ATCInput(flight_id=flight_id)),
            crew_logic(CrewInput(flight_id=flight_id)),
            mx_logic(MaintenanceInput(flight_id=flight_id)),
            fleet_logic(FleetInput(flight_id=flight_id))
        )

        # Step 2: Decision Logic
        if not c["ready"]:
            rec = await recovery_logic(RecoveryInput(flight_id=flight_id, requirement="CREW"))
            decision, reason = ("DELAY", "Crew fatigue mitigated by Standby") if rec["success"] else ("CANCEL", f"Crew fatigued ({c['hours_left']}h)")
        elif m["status"] != "GO":
            decision, reason = ("SWAP AIRCRAFT", f"Tail {m['status']} -> swapped to {f['sub_tail']}") if f["spare_ready"] else ("CANCEL", f"Maint issue ({m['status']}) & no spare")
        elif w["severity"] == "HIGH" or a["delay"] >= 60:
            decision, reason = "DELAY", f"Holding for {w['status']} / ATC"
        else:
            decision, reason = "PROCEED", "Nominal conditions"

        # Step 3: Cost and Telemetry
        cost_rep = await cost_logic(CostInput(flight_id=flight_id, decision=decision, delay_minutes=a["delay"], pax_count=p["count"]))
        latency = (time.perf_counter() - start_time) * 1000
        await obs_logic(ObservabilityInput(flight_id=flight_id, trace={"decision": decision}, latency_ms=latency))

        final = await refiner_logic(RefinerInput(decision=decision, reason=reason, flight_id=flight_id, cost=cost_rep["total_cost"]))
        return {"id": flight_id, "decision": decision, "cost": cost_rep["total_cost"], "summary": final["notification"]}

# 5. Bulk Simulation
async def run_fleet_simulation(flight_count: int = 5):
    print("═" * 60 + f"\n     NVIDIA NAT • FULL OBSERVABILITY FLEET DEMO\n" + "═" * 60)
    results = await asyncio.gather(*[run_full_aocc(f"NV-{2026+i}") for i in range(flight_count)])
    for r in results: print(f"[{r['id']}] {r['summary']}")
    print("─" * 60 + f"\nTotal Fleet Cost: ${sum(r['cost'] for r in results):,}\n" + "─" * 60)

await run_fleet_simulation(5)

Function registry cleared
════════════════════════════════════════════════════════════
     NVIDIA NAT • FULL OBSERVABILITY FLEET DEMO
════════════════════════════════════════════════════════════
[OBSERVABILITY] NV-2026 → Latency: 7.65ms
[OBSERVABILITY] NV-2027 → Latency: 7.62ms
[OBSERVABILITY] NV-2028 → Latency: 7.59ms
[OBSERVABILITY] NV-2029 → Latency: 7.55ms
[OBSERVABILITY] NV-2030 → Latency: 7.51ms
[NV-2026] CRITICAL: Flight CANCELLED. Maint issue (DEFERRED) & no spare. Cost: $36,900
[NV-2027] CRITICAL: Flight CANCELLED. Maint issue (AOG) & no spare. Cost: $30,600
[NV-2028] OPS ALERT: Aircraft SWAP. Tail MEL -> swapped to N612NV. Cost: $5,000
[NV-2029] SCHEDULE UPDATE: Flight DELAYED. Holding for STRONG_WINDS / ATC. Cost: $4,545
[NV-2030] CRITICAL: Flight CANCELLED. Maint issue (MEL) & no spare. Cost: $37,950
────────────────────────────────────────────────────────────
Total Fleet Cost: $114,995
────────────────────────────────────────────────────────────
